In [2]:
# Use a pipeline as a high-level helper
from transformers import pipeline
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("sarvamai/OpenHathi-7B-Hi-v0.1-Base")
model = AutoModelForCausalLM.from_pretrained("sarvamai/OpenHathi-7B-Hi-v0.1-Base")
pipe = pipeline("text-generation", model="sarvamai/OpenHathi-7B-Hi-v0.1-Base")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
# Function to query the openHathi model multiple times for a single prompt
def query_model(prompt, num_responses=1):
    # Generate multiple responses for the same prompt
    responses = pipe(prompt, max_length=100, num_return_sequences=num_responses)
    
    # Collect all generated responses
    generated_texts = [response['generated_text'] for response in responses]
    return generated_texts

# Function to test hallucination by collecting multiple responses for each prompt
def test_hallucinations(prompts, num_responses=1):
    print("Testing for Hallucinations (Self-Consistency):")
    hallucination_results = {}
    
    for i, prompt in enumerate(prompts, 1):
        # Generate variations of the prompt to get diverse responses
        variations = [
            f"{prompt} Can you elaborate on that?",
            f"{prompt} Explain in a laymann's term?",
        ]
        
        # Collect responses for each variation
        all_responses = []
        for variation in variations:
            responses = query_model(variation, num_responses=num_responses)
            all_responses.extend(responses)
        
        # Print responses for each variation
        print(f"Prompt {i}: {prompt}")
        print("Responses:")
        
        for j, response in enumerate(all_responses, 1):
            print(f"Response {j}: {response}\n")
        
        hallucination_results[prompt] = all_responses
        
    return hallucination_results

# Function to query the openHathi model using the pipeline (unchanged)
def query_model_single(prompt):
    response = pipe(prompt, max_length=100, num_return_sequences=1)
    return response[0]['generated_text']

# Function to test fact-checking (unchanged)
def test_fact_checking(facts):
    print("Testing Fact-Checking:")
    fact_checks = {}
    for fact, expected_result in facts.items():
        response = query_model_single(fact)
        print(f"Fact: {fact}")
        print(f"Model's Response: {response}")
        print(f"Expected Result: {expected_result}")
        is_correct = response.lower() == expected_result.lower()
        print(f"Is Correct: {is_correct}\n")
        fact_checks[fact] = {"response": response, "expected": expected_result, "correct": is_correct}
    return fact_checks

if __name__ == "__main__":
    # Self-consistency (hallucination detection) prompts
    consistency_prompts = [
        "What is the best way to treat common cold?",
        "Is 100.001 greater than 100.? If so, why?",
        "What do you mean by queen's necklace...is it Mumbai's famous place or something else?"
    ]
    
    # Test for hallucinations (self-consistency)
    hallucination_results = test_hallucinations(consistency_prompts, num_responses=2)

    # Fact-checking prompts and expected results
    fact_check_prompts = {
        "Albert Einstein won the Nobel Prize for his theory of relativity.": "Correct",
        "The smallest mountain in the world is Mount Everest.": "Incorrect",
        "Python was developed by Guido van Rossum in the 1990s.": "Partially correct"
    }
    
    # Test fact-checking
    fact_check_results = test_fact_checking(fact_check_prompts)


Testing for Hallucinations (Self-Consistency):
Prompt 1: What is the best way to treat common cold?
Responses:
Response 1: What is the best way to treat common cold? Can you elaborate on that?
---
सामान्य सर्दी के इलाज के लिए सबसे अच्छा तरीका लक्षणों को प्रबंधित करना और ठीक होने में तेजी लाने के लिए उचित देखभाल करना है। Here are some tips:

1. हाइड्रेटेड रहेंः बहुत सारा पानी पीएँ, क्योंकि निर्जलीकरण सर्दी के लक्षणों को खराब कर सकता है।

2. Rest: Rest your body and allow it to recover.

Response 2: What is the best way to treat common cold? Can you elaborate on that?
---
सामान्य सर्दी के इलाज के लिए सबसे अच्छा तरीका लक्षणों को प्रबंधित करना और ठीक होने में तेजी लाने के लिए उचित देखभाल करना है। Here are some tips:

1. हाइड्रेटेड रहेंः बहुत सारा पानी पीएँ, क्योंकि निर्जलीकरण सर्दी के लक्षणों को खराब कर सकता है।

2. Rest: Rest your body and allow it to recover.

Response 3: What is the best way to treat common cold? Explain in a laymann's term?
---
Common cold is a viral infection that aff

-------------------------------------------------------------------------------------------------------------------
Using RAG to augment the correct generation of text for the openHathi LLM model
-------------------------------------------------------------------------------------------------------------------

In [6]:
#New RAG implementation

!pip install langchain
!pip install git+https://github.com/huggingface/transformers.git
!pip install git+https://github.com/huggingface/accelerate.git
!pip install sentence-transformers==2.2.2
!pip install pinecone-client
!pip install bitsandbytes
!pip install datasets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-gfubrt6j
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-gfubrt6j
  Resolved https://github.com/huggingface/transformers.git to commit cea9ec086a14da1320940c5e48d7bd5dbcf32734
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Cloning https://github.com/huggingface/accelerate.git to /tmp/pip-req-build-y9pughya
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate.git /tmp/pip-req-build-y9pughya
  Resolved https://github.com/huggingface/accelerate.git to commit fc52fa969ea7a7f01d31c535717be9b5f72351e2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached sentence_transformers-2.2.2-py3-none-any.whl
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 3.1.0
    Uninstalling sentence-transformers-3.1.0:
      Successfully uninstalled sentence-transformers-3.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-huggingface 0.0.3 requires sentence-transformers>=2.6.0, but you have sentence-transformers 2.2.2 which is incompatible.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
import os

# Function to load all text files from the 'rag' folder into a list
def load_knowledge_base_from_folder(folder_path='rag'):
    knowledge_base = []
    
    for filename in os.listdir(folder_path):
        if filename.endswith('.txt'):
            file_path = os.path.join(folder_path, filename)
            print(file_path)
            
            # Read the content of the file
            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read()
                knowledge_base.append({"id": filename, "text": content})
    
    return knowledge_base

# Load the knowledge base from the folder
knowledge_base = load_knowledge_base_from_folder()


In [ ]:
import os
from pinecone import Pinecone, ServerlessSpec

# Initialize the Pinecone instance
pc = Pinecone(api_key='56b5d916-78bb-48a6-a153-7c371f9acc0a')

# Check if the index already exists, if not, create it
index_name = 'rag-assignment'
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric='cosine',
        spec=ServerlessSpec(cloud='aws', region='us-east-1')
    )

# Retrieve the index
index = pc.Index(index_name)
index.describe_index_stats()

# Rest of your code remains unchanged...


In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

# Load embedding model
embedding_model_id = 'sentence-transformers/all-MiniLM-L6-v2'

embedding_model = HuggingFaceEmbeddings(
    model_name=embedding_model_id,
)

batch_size = 2
for i in range(0, len(knowledge_base), batch_size):
    i_end = min(len(knowledge_base), i + batch_size)
    batch = knowledge_base[i:i_end]
    
    # Prepare IDs and texts
    ids = [doc['id'] for doc in batch]
    texts = [doc['text'] for doc in batch]

    # Get embeddings for the texts
    embeddings = embedding_model.embed_documents(texts)  # Convert to list

    # Prepare metadata
    meta_data = [{
        'text': doc['text'],
    } for doc in batch]
    
    # Prepare vectors for upsert
    vectors = [(id_, embedding, meta) for id_, embedding, meta in zip(ids, embeddings, meta_data)]
    
    # Upsert to Pinecone
    index.upsert(vectors=vectors)

print("Data upserted successfully.")
index.describe_index_stats()
